In [1]:
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

import logging

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import (Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, 
                                     BatchNormalization, GlobalAveragePooling2D)
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
import datetime


In [2]:
df = pd.read_csv("../data/raw/HAM10000_metadata.csv")
df.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear


In [3]:
df.dx.value_counts()

dx
nv       6705
mel      1113
bkl      1099
bcc       514
akiec     327
vasc      142
df        115
Name: count, dtype: int64

### Clasificación de lesiones de piel por tipo (`dx`)

En conjuntos de datos dermatológicos como HAM10000, las lesiones se clasifican en siete tipos según el campo `dx`. A continuación se indica cuáles se consideran benignas y cuáles malignas:

#### Lesiones benignas
- **`nv` (Nevus melanocítico)**: Lunares comunes, generalmente benignos.
- **`bkl` (Queratosis benigna)**: Incluye lentigos solares y queratosis seborreicas.
- **`df` (Dermatofibroma)**: Tumores benignos de la piel, firmes y pequeños.
- **`vasc` (Lesiones vasculares)**: Incluye angiomas y otras lesiones vasculares, consideradas benignas.

#### Lesiones malignas
- **`mel` (Melanoma)**: Cáncer de piel agresivo y potencialmente mortal.
- **`bcc` (Carcinoma basocelular)**: Cáncer de piel de crecimiento lento pero invasivo.
- **`akiec` (Queratosis actínica / carcinoma intraepidérmico)**: Lesión precancerosa o carcinoma in situ.

Esta clasificación permite agrupar las clases en dos categorías para tareas de clasificación binaria: benignas vs malignas.


In [4]:
data_dir = "../data/processed/train"

# Augmentación para entrenamiento
datagen = ImageDataGenerator(
    rescale=1./255, 
    rotation_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    brightness_range=[0.8,1.2],
    shear_range=0.2,
    horizontal_flip=True, # Contempla manchas simétricas
    validation_split=0.15,  # Separación interna
)

# Generador de entrenamiento
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='training',
    shuffle=True
)

val_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation',
    shuffle=False
)

print(pd.Series(val_generator.classes).value_counts())
print(pd.Series(train_generator.classes).value_counts())

Found 7662 images belonging to 2 classes.
Found 1351 images belonging to 2 classes.
0    1088
1     263
Name: count, dtype: int64
0    6166
1    1496
Name: count, dtype: int64


In [5]:
labels = train_generator.classes

# Calculamos los pesos
class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(labels),
    y=labels
)

# Lo convertimos en diccionario para Keras
class_weights = dict(enumerate(class_weights))
print(class_weights)


{0: 0.6213104119364256, 1: 2.560828877005348}


## Modelo

In [6]:
def clasificador_binario(input_shape=(224,224,3), lr=1e-3):
    entrada = Input(shape=input_shape, name='entrada_imagen')

    # Bloque 1
    x = Conv2D(32, (3,3), activation='relu', padding='same', kernel_regularizer=l2(1e-4))(entrada)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2,2))(x)

    # Bloque 2
    x = Conv2D(64, (3,3), activation='relu', padding='same', kernel_regularizer=l2(1e-4))(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2,2))(x)

    # Bloque 3
    x = Conv2D(128, (3,3), activation='relu', padding='same', kernel_regularizer=l2(1e-4))(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2,2))(x)

    # Bloque 4 (extra para más capacidad)
    x = Conv2D(256, (3,3), activation='relu', padding='same', kernel_regularizer=l2(1e-4))(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2,2))(x)

    # Global pooling en lugar de Flatten (reduce parámetros)
    x = GlobalAveragePooling2D()(x)

    # Capa densa
    x = Dense(128, activation='relu', kernel_regularizer=l2(1e-4))(x)
    x = Dropout(0.5)(x)

    salida = Dense(1, activation='sigmoid', name='salida_binaria')(x)

    modelo = Model(inputs=entrada, outputs=salida, name='cnn_mejorada')
    modelo.compile(optimizer=Adam(learning_rate=lr),
                   loss='binary_crossentropy',
                   metrics=['accuracy'])
    return modelo



In [7]:
import tensorflow as tf

# Desactiva todas las GPUs
tf.config.set_visible_devices([], 'GPU')


W0000 00:00:1763631493.891450    6761 gpu_device.cc:2431] TensorFlow was not built with CUDA kernel binaries compatible with compute capability 5.2. CUDA kernels will be jit-compiled from PTX, which could take 30 minutes or longer.


In [ ]:
# Ajustar nivel de logging de TensorFlow
logging.getLogger("tensorflow").setLevel(logging.ERROR)

# modelo = clasificador_binario()  
modelo = load_model("../models/classifier/new_model_11_19_h21_50.keras")

history = modelo.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20,
    class_weight=class_weights
)


Epoch 1/20


2025-11-20 10:38:50.889106: I external/local_xla/xla/service/service.cc:163] XLA service 0x7199b40148c0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2025-11-20 10:38:50.889124: I external/local_xla/xla/service/service.cc:171]   StreamExecutor device (0): Host, Default Version
2025-11-20 10:38:51.005234: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1763631532.845834    8056 device_compiler.h:196] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
2025-11-20 10:38:52.848915: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 2667125736 exceeds 10% of free system memory.


  1/240 ━━━━━━━━━━━━━━━━━━━━ 28:12 7s/step - accuracy: 0.6875 - loss: 0.6500

2025-11-20 10:38:54.754612: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 2667125736 exceeds 10% of free system memory.


  2/240 ━━━━━━━━━━━━━━━━━━━━ 7:31 2s/step - accuracy: 0.7109 - loss: 0.6003 

...

  3/240 ━━━━━━━━━━━━━━━━━━━━ 7:13 2s/step - accuracy: 0.7101 - loss: 0.5665

2025-11-20 10:38:58.410754: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 2667125736 exceeds 10% of free system memory.


  4/240 ━━━━━━━━━━━━━━━━━━━━ 7:44 2s/step - accuracy: 0.7103 - loss: 0.5560

2025-11-20 10:39:00.656839: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 2667125736 exceeds 10% of free system memory.


240/240 ━━━━━━━━━━━━━━━━━━━━ 483s 2s/step - accuracy: 0.7198 - loss: 0.4936 - val_accuracy: 0.7128 - val_loss: 0.6005
Epoch 2/20
240/240 ━━━━━━━━━━━━━━━━━━━━ 474s 2s/step - accuracy: 0.7163 - loss: 0.4926 - val_accuracy: 0.7979 - val_loss: 0.4024
Epoch 3/20
240/240 ━━━━━━━━━━━━━━━━━━━━ 475s 2s/step - accuracy: 0.7167 - loss: 0.4880 - val_accuracy: 0.7742 - val_loss: 0.5640
Epoch 4/20
240/240 ━━━━━━━━━━━━━━━━━━━━ 477s 2s/step - accuracy: 0.7313 - loss: 0.4890 - val_accuracy: 0.6736 - val_loss: 0.6230
Epoch 5/20
240/240 ━━━━━━━━━━━━━━━━━━━━ 477s 2s/step - accuracy: 0.7143 - loss: 0.4868 - val_accuracy: 0.7631 - val_loss: 0.5244
Epoch 6/20
240/240 ━━━━━━━━━━━━━━━━━━━━ 477s 2s/step - accuracy: 0.7257 - loss: 0.4807 - val_accuracy: 0.7927 - val_loss: 0.3966
Epoch 7/20
240/240 ━━━━━━━━━━━━━━━━━━━━ 479s 2s/step - accuracy: 0.7173 - loss: 0.4808 - val_accuracy: 0.8216 - val_loss: 0.4995
Epoch 8/20
240/240 ━━━━━━━━━━━━━━━━━━━━ 480s 2s/step - accuracy: 0.7300 - loss: 0.4796 - val_accuracy: 0.786

In [ ]:
# Current timestamp
timestamp = datetime.datetime.now().strftime("%m_%d_h%H_%M")

# Carpeta donde guardar
save_dir = "../models/classifier"
os.makedirs(save_dir, exist_ok=True)

modelo.save(f"../models/classifier/new_model_{timestamp}.keras")

In [ ]:
modelo.summary()

Model: "cnn_mejorada"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ entrada_imagen (InputLayer)     │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 224, 224, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 224, 224, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 112, 112, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 112, 112, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 56, 56, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 56, 56, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 28, 28, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 28, 28, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 14, 14, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 256)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ salida_binaria (Dense)          │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,268,165 (4.84 MB)

 Trainable params: 422,401 (1.61 MB)

 Non-trainable params: 960 (3.75 KB)

 Optimizer params: 844,804 (3.22 MB)